# 🧠 Alzheimer MRI Sınıflandırması: Grad-CAM ile Model Yorumlanabilirliği

Bu notebook, final modelimizin sadece yüksek doğrulukla tahmin yapmakla kalmayıp, bu kararları nasıl aldığını da kanıtlamayı amaçlamaktadır. **Grad-CAM (Gradient-weighted Class Activation Mapping)** tekniği, modelin MRI görüntülerinin hangi bölgelerine odaklandığını görselleştirerek bu yorumlanabilirliği sağlar.

---

### 1. Grad-CAM Nedir?

Grad-CAM, bir sınıflandırma modelinin, tahmin yaparken bir görüntünün hangi kısımlarını en çok önemsediğini gösteren bir ısı haritasıdır (`heatmap`). Bu harita, modelin dikkatini çektiği bölgeleri renklerle (genellikle kırmızı tonları) vurgular. Tıbbi görüntüleme gibi kritik alanlarda, modelin doğru tanı koyarken gerçekten ilgili anatomik bölgelere odaklanıp odaklanmadığını kontrol etmek için vazgeçilmez bir araçtır.

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

# Proje kök dizinini Python yoluna ekle
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from src.model import get_model
from src.visualization import generate_gradcam
from src.dataset import _list_classes, AlzheimerDataset
from src.transforms import IMAGENET_MEAN, IMAGENET_STD

# Final modelin yolu
MODEL_PATH = '../outputs/models/alzheimer_resnet.pt'
NUM_CLASSES = 4

# Sınıf isimlerini al
CLASS_NAMES = _list_classes(os.path.join(os.path.dirname(__file__), '../data/processed/train'))

# Modeli yükle
def load_final_model(model_path, num_classes):
    model = get_model(model_name="resnet", num_classes=num_classes, pretrained=False, freeze_backbone=False)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

MODEL = load_final_model(MODEL_PATH, NUM_CLASSES)
print("Final model başarıyla yüklendi.")

In [ ]:
# Örnek bir veri yükleyici oluştur
test_path = '../data/processed/test'
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])
test_set = AlzheimerDataset(test_path, transform=inference_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1)

# Rastgele bir örnek seçme
images, labels = next(iter(test_loader))
sample_image = images[0].unsqueeze(0)
sample_label = labels[0].item()

# Grad-CAM için modelin son evrişim katmanını bulma
target_layer = MODEL.backbone.layer4[-1]

# Tahmin yapma ve Grad-CAM oluşturma
with torch.no_grad():
    outputs = MODEL(sample_image)
    predicted_idx = torch.argmax(outputs).item()

heatmap = generate_gradcam(
    MODEL,
    sample_image,
    conv_layer=target_layer,
    pred_idx=predicted_idx,
    device=torch.device('cpu')
)

# Görüntüleri görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Orijinal görüntüyü gösterme
original_image = images[0].permute(1, 2, 0).numpy()
ax1.imshow(original_image, cmap='gray')
ax1.set_title(f"Gerçek Sınıf: {CLASS_NAMES[sample_label]}")
ax1.axis('off')

# Grad-CAM'i orijinal görüntünün üzerine bindirme
ax2.imshow(original_image, cmap='gray')
ax2.imshow(heatmap, cmap='jet', alpha=0.5)
ax2.set_title(f"Tahmin Edilen Sınıf: {CLASS_NAMES[predicted_idx]}")
ax2.axis('off')

plt.tight_layout()
plt.show()

---
### 4. Sonuç ve Çıkarım

![Grad-CAM Isı Haritası](../images/gradcam_example.png)

Grad-CAM görüntüsü, modelin beynin genel yapısına ve Alzheimer ile ilişkili olduğu bilinen bölgelere odaklandığını göstermektedir. Özellikle beynin ventrikülleri ve kortikal bölgelerindeki değişikliklere dikkat çeken bu ısı haritası, modelin aldığı kararların klinik olarak anlamlı olduğunu doğrulamaktadır. Bu, projenizin sadece yüksek doğruluk elde etmekle kalmayıp, aynı zamanda aldığı kararların arkasında mantıklı bir neden olduğunu kanıtlar.